In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/AI Masters/Data Visualization/archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Indicators.csv')

## Restructure dataset

In [ ]:
col_map = {'Country': [], 'Year': []}

for col_name in df['IndicatorName'].unique():
    col_map[col_name] = []

current_country, years = df['CountryName'][0], []

for key, row in df.iterrows():
    if current_country != row['CountryName']:
        years = []
        current_country = row['CountryName']
    if row['Year'] not in years:
        years.append(row['Year'])
        print(row['CountryName'], row['Year'])
        df_temp = df[df['CountryName'] == row['CountryName']][df['Year'] == row['Year']]
        
        col_map['Country'].append(row['CountryName'])
        col_map['Year'].append(row['Year'])

        for key_temp, row_temp in df_temp.iterrows():
            col_map[row_temp['IndicatorName']].append(row_temp['Value'])
        
        for col_name in df['IndicatorName'].unique():
            if len(col_map[col_name]) == len(col_map['Country']) - 1:
                col_map[col_name].append(np.nan)

df1 = pd.DataFrame(col_map)
df1.to_csv('/content/drive/MyDrive/AI Masters/Data Visualization/Indicators.csv', index=False)

## Crude EDA stump

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AI Masters/Data Visualization/Indicators.csv')

In [ ]:
no_categorical = 0
for column in df.columns:
    if '(1=low to 6=high)' in column or '(1-5 scale; 5=best)' in column:
        print(column)
        no_categorical += 1
print(no_categorical)

CPIA building human resources rating (1=low to 6=high)
CPIA business regulatory environment rating (1=low to 6=high)
CPIA debt policy rating (1=low to 6=high)
CPIA economic management cluster average (1=low to 6=high)
CPIA efficiency of revenue mobilization rating (1=low to 6=high)
CPIA equity of public resource use rating (1=low to 6=high)
CPIA financial sector rating (1=low to 6=high)
CPIA fiscal policy rating (1=low to 6=high)
CPIA gender equality rating (1=low to 6=high)
CPIA macroeconomic management rating (1=low to 6=high)
CPIA policies for social inclusion/equity cluster average (1=low to 6=high)
CPIA policy and institutions for environmental sustainability rating (1=low to 6=high)
CPIA property rights and rule-based governance rating (1=low to 6=high)
CPIA public sector management and institutions cluster average (1=low to 6=high)
CPIA quality of budgetary and financial management rating (1=low to 6=high)
CPIA quality of public administration rating (1=low to 6=high)
CPIA socia

In [ ]:
for column in df.columns:
    good = True
    for token in ['$', 'GNI', 'GDP', 'LCU', 'expenditure', 'health', 'Health', 'death', 'hospital', 'export', 'import', 'Export', 'Import']:
        if token in column:
            good = False
    if good:
        print(column)

Country
Year
Adolescent fertility rate (births per 1,000 women ages 15-19)
Age dependency ratio (% of working-age population)
Age dependency ratio, old (% of working-age population)
Age dependency ratio, young (% of working-age population)
Birth rate, crude (per 1,000 people)
CO2 emissions (kt)
CO2 emissions (metric tons per capita)
CO2 emissions from gaseous fuel consumption (% of total)
CO2 emissions from liquid fuel consumption (% of total)
CO2 emissions from liquid fuel consumption (kt)
CO2 emissions from solid fuel consumption (% of total)
Death rate, crude (per 1,000 people)
Fertility rate, total (births per woman)
Fixed telephone subscriptions
Fixed telephone subscriptions (per 100 people)
Hospital beds (per 1,000 people)
International migrant stock (% of population)
International migrant stock, total
Life expectancy at birth, female (years)
Life expectancy at birth, male (years)
Life expectancy at birth, total (years)
Mobile cellular subscriptions
Mobile cellular subscriptions 

## Data Imputation

In [ ]:
from fancyimpute import KNN, BiScaler, SoftImpute
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/AI Masters/Data Visualization/Indicators.csv')

file1 = open('selection.txt', 'r')
lines = file1.readlines()
selected_features = ['Country', 'Year']

for line in lines:
    if line.strip() in df.columns:
        selected_features.append(line.strip())

file1.close()

In [ ]:
df_selection = df[selected_features]

In [ ]:
country_names = df_selection['Country']
years = df_selection['Year']

df_np = df_selection.drop(labels = ['Country'], axis = 1).select_dtypes(include=[np.float, np.int]).values

df_incomplete_normalized = BiScaler().fit_transform(df_np)
df_softimpute = SoftImpute().fit_transform(df_incomplete_normalized)

[BiScaler] Initial log residual value = 61.562127
[BiScaler] Iter 1: log residual = 7.952083, log improvement ratio=53.610043
[BiScaler] Iter 2: log residual = 7.396630, log improvement ratio=0.555453
[BiScaler] Iter 3: log residual = 6.887622, log improvement ratio=0.509008
[BiScaler] Iter 4: log residual = 6.425015, log improvement ratio=0.462607
[BiScaler] Iter 5: log residual = 5.965654, log improvement ratio=0.459362
[BiScaler] Iter 6: log residual = 5.523957, log improvement ratio=0.441696
[BiScaler] Iter 7: log residual = 5.115078, log improvement ratio=0.408879
[BiScaler] Iter 8: log residual = 4.760949, log improvement ratio=0.354129
[BiScaler] Iter 9: log residual = 4.476920, log improvement ratio=0.284029
[BiScaler] Iter 10: log residual = 4.266632, log improvement ratio=0.210288
[BiScaler] Iter 11: log residual = 4.122112, log improvement ratio=0.144521
[BiScaler] Iter 12: log residual = 4.028742, log improvement ratio=0.093370
[BiScaler] Iter 13: log residual = 3.971164, l

In [ ]:
df_fn = pd.DataFrame(data = df_softimpute, columns = df_selection.columns[1:])

df_fn.drop(labels = ['Year'], axis = 1, inplace = True)

df_fn.insert(0, 'CountryName', country_names)

df_fn.insert(1, 'Year', years)

In [ ]:
df_fn

,CountryName,Year,GDP per capita (current US$),GDP per capita growth (annual %),GNI (current US$),GNI per capita growth (annual %),Health expenditure per capita (current US$),Population density (people per sq. km of land area),Population growth (annual %),Urban population,Rural population,Improved sanitation facilities (% of population with access),Improved water source (% of population with access),Merchandise exports (current US$),Merchandise imports (current US$),Exports of goods and services (annual % growth),Crop production index (2004-2006 = 100),Research and development expenditure (% of GDP),Natural gas rents (% of GDP),Debt stock reduction (current US$),General government final consumption expenditure (current US$),Exports of goods and services (current US$),Imports of goods and services (current US$),Trade (% of GDP),Arms exports (SIPRI trend indicator values),Arms imports (SIPRI trend indicator values),Interest payments on external debt (% of GNI),Oil rents (% of GDP),Coal rents (% of GDP),External balance on goods and services (% of GDP),Gross capital formation (current US$),Gross domestic savings (current US$),Population growth (annual %),"Population, total",Prevalence of anemia among children (% of children under 5),Prevalence of anemia among non-pregnant women (% of women ages 15-49),Prevalence of anemia among pregnant women (%),"Prevalence of HIV, female (% ages 15-24)","Prevalence of HIV, male (% ages 15-24)","Prevalence of HIV, total (% of population ages 15-49)",Children (0-14) living with HIV,"Incidence of tuberculosis (per 100,000 people)",Diabetes prevalence (% of population ages 20 to 79),"Immunization, DPT (% of children ages 12-23 months)","Immunization, measles (% of children ages 12-23 months)","Maternal mortality ratio (modeled estimate, per 100,000 live births)","Maternal mortality ratio (national estimate, per 100,000 live births)","Mortality rate, adult, female (per 1,000 female adults)","Mortality rate, adult, male (per 1,000 male adults)","Mortality rate, infant (per 1,000 live births)","Mortality rate, infant, female (per 1,000 live births)","Mortality rate, infant, male (per 1,000 live births)","Mortality rate, neonatal (per 1,000 live births)","Mortality rate, under-5, female (per 1,000 live births)","Mortality rate, under-5, male (per 1,000 live births)","Cause of death, by injury (% of total)","Death rate, crude (per 1,000 people)",Number of infant deaths,Number of maternal deaths,Number of neonatal deaths,Number of under-five deaths,Exclusive breastfeeding (% of children under 6 months),"Condom use, population ages 15-24, female (% of females ages 15-24)","Condom use, population ages 15-24, male (% of males ages 15-24)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant),"Tuberculosis case detection rate (%, all forms)",Tuberculosis treatment success rate (% of new cases),"Adolescent fertility rate (births per 1,000 women ages 15-19)",Female headed households (% of households with a female head),Newborns protected against tetanus (%)
0,Arab World,1960,-0.045593,-0.408164,0.263350,-0.327026,-0.141904,0.091056,-1.013009,1.016292,1.864989,-0.196346,-0.448973,-0.117691,-0.117335,-0.043567,-0.363045,-0.664290,-0.629361,0.151779,0.150987,0.042193,0.042590,-0.281923,-0.400485,0.352656,-0.854541,-0.050959,-1.061955,0.162951,0.347518,0.334454,-1.013009,1.625104,-0.030946,0.090783,-0.260072,-0.444013,-0.705244,-0.308377,0.235815,0.329762,-0.287659,-0.084173,-0.055859,0.035783,0.108761,0.243339,0.199433,0.730655,0.105161,0.159534,-0.034689,0.131383,0.160323,-0.219152,-1.549895,1.214998,0.674885,1.230199,1.099615,-0.091827,-0.116362,-0.122448,-0.450302,-1.756447,-0.336532,-0.217747,-0.156285,-0.146829,0.155244,-0.163832,-0.173464
1,Caribbean small states,1960,-0.747799,0.390578,-0.773346,0.351103,-0

In [80]:
df_fn.to_csv('/content/drive/MyDrive/AI Masters/Data Visualization/IndicatorsV2.csv')